In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Read me:

- The minUserR and minBookR will be used to ensure statistical significance, please set to 0 if not required. Here I will set to 200, 100 as suggested in the example.
- To get the the N recommendations for each user, we will get the similarity for each book and remaining books and get the highest N
- Here, we have the similarity for each book with all the books rated 10, then we have have N ratings for each book.I though of 2 ways to deal with this:
<ul>
<li>Average Rating Technique, Get the average for each book similarity from all N, and use this as an indicator, but the problem with this apprach if we have a book too similar to 1 book and has low similarity with other books will be neglected only books that are similar to all books will be considered which may results in book too far from each highly rated books. fn "getTopNRecommentationsforUserAvg"</li>
<li>Individual Rating Technique, Keep each similarity for each book and order total list, that will solve the problem mentioned above in case the user has highly rated books in different domains the recommender will return result for each book . fn "getTopNRecommentationsforUser"</li>
</ul>
- I will check the result from each system and compare the rating of the user "which will not be considered in the system due to minimum ratings filter" to the recommendations to see which is better.

In [2]:
books = pd.read_csv('BX-Books.csv', sep=';', error_bad_lines=False, warn_bad_lines=False, encoding="latin-1")
books.columns = ['ISBN', 'bookTitle', 'bookAuthor', 'yearOfPublication', 'publisher', 'imageUrlS', 'imageUrlM', 'imageUrlL']
users = pd.read_csv('BX-Users.csv', sep=';', error_bad_lines=False, warn_bad_lines=False, encoding="latin-1")
users.columns = ['userID', 'Location', 'Age']
ratingsOriginal = pd.read_csv('BX-Book-Ratings.csv', sep=';', error_bad_lines=False, warn_bad_lines=False, encoding="latin-1")
ratingsOriginal.columns = ['userID', 'ISBN', 'bookRating']
ratings = pd.read_csv('BX-Book-Ratings.csv', sep=';', error_bad_lines=False, warn_bad_lines=False, encoding="latin-1")
ratings.columns = ['userID', 'ISBN', 'bookRating']

C:\Users\be231\AppData\Local\Continuum\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
minUserR = 200 # minimum number of rating per user

In [4]:
minBookR = 100 # minimum number of rating per book

In [5]:
minRating = 10 # minimum rating value for book to get the similar books

In [6]:

countUserR = ratings['userID'].value_counts()
ratings = ratings[ratings['userID'].isin(countUserR[countUserR >= minUserR].index)]
countBookR = ratings['ISBN'].value_counts()
ratings = ratings[ratings['ISBN'].isin(countBookR[countBookR >= minBookR].index)]

In [7]:
ratings_pivot = ratings.pivot(index='userID', columns='ISBN', values='bookRating')

In [8]:
def getTopNRecommentationsforUserAvg(userID,N,minRating,ratings_pivot):
    #get books rated "minRating" by the user
    topBookUser = ratings[(ratings['userID'] == userID) & (ratings['bookRating'] >= minRating)].sort_values('bookRating', ascending=False)
    topRatedBooksUser = []
    bookSimList = []
    #for each book get the similarities with other books
    for bookISBN in topBookUser['ISBN']:
        #print(bookISBN)
        topRatedBooksUser.append(bookISBN)
        book_ratings = ratings_pivot[bookISBN]
        bookSimilarities = ratings_pivot.corrwith(book_ratings)
        #print(bookSimilarities.head())
        corr_bones = pd.DataFrame(bookSimilarities, columns=['pearsonR'])
        corr_bones.dropna(inplace=True)
        corr_bones = corr_bones.sort_values('pearsonR', ascending=False)
        #print(corr_bones.head())
        #print(corr_bones.columns)
        bookSimList.append(corr_bones)# save the similarities for each book
        
    bookSimComb = pd.concat(bookSimList)# create 1 df from all similarities of all books
    bookSimComb = bookSimComb.groupby(bookSimComb.index).mean()
    bookSimComb = bookSimComb.sort_values('pearsonR', ascending=False)
    #print(bookSimComb.columns)
    #print(bookSimComb.head())
    topNbooks = bookSimComb.head(N+len(topRatedBooksUser))
    topNbooksList = list(topNbooks.drop('pearsonR',axis=1).index)
    x = []
    for book in topNbooksList:
        if (book not in topRatedBooksUser):
            x.append(book)
    return x

In [9]:
def getTopNRecommentationsforUser(userID,N,minRating,ratings_pivot):
    #get books rated "minRating" by the user
    topBookUser = ratings[(ratings['userID'] == userID) & (ratings['bookRating'] >= minRating)].sort_values('bookRating', ascending=False)
    topRatedBooksUser = []
    bookSimList = []
    #for each book get the similarities with other books
    for bookISBN in topBookUser['ISBN']:
        #print(bookISBN)
        topRatedBooksUser.append(bookISBN)
        book_ratings = ratings_pivot[bookISBN]
        bookSimilarities = ratings_pivot.corrwith(book_ratings)
        #print(bookSimilarities.head())
        corr_bones = pd.DataFrame(bookSimilarities, columns=['pearsonR'])
        corr_bones.dropna(inplace=True)
        corr_bones = corr_bones.sort_values('pearsonR', ascending=False)
        #print(corr_bones.head())
        bookSimList.append(corr_bones)# save the similarities for each book
        
    bookSimComb = pd.concat(bookSimList)# create 1 df from all similarities of all books
    bookSimComb = bookSimComb.sort_values('pearsonR', ascending=False)
    #print(bookSimComb.columns)
    #print(bookSimComb.head())
    topNbooks = bookSimComb.head(N+len(topRatedBooksUser))
    return list(topNbooks.drop('pearsonR',axis=1).index)[len(topRatedBooksUser):]

In [10]:
N = 30

In [11]:
userID = 3363
topRecommendations = getTopNRecommentationsforUser(userID,N,minRating,ratings_pivot)
print(topRecommendations)

print (ratingsOriginal[(ratingsOriginal['userID'] == userID) & ratingsOriginal['ISBN'].isin(topRecommendations)])

['0385265700', '0446611212', '0440236673', '0440241073', '0446611212', '0385484518', '0345378490', '0345391055', '0446610038', '0440221471', '0345370775', '0553280341', '0345361792', '080410753X', '044651652X', '0671510053', '0385492081', '0316666343', '0553572997', '0345353145', '0553268880', '014028009X', '0312278586', '0446606812', '0440214041', '080410753X', '0449212602', '0446608955', '044022165X', '0440234743']
       userID        ISBN  bookRating
18007    3363  0312278586           0
18025    3363  0316666343           0
18229    3363  0440234743           0
18230    3363  0440236673           0
18254    3363  0446606812           0


In [12]:
print(books[books['ISBN'].isin(topRecommendations)]["bookTitle"])

18                                          The Testament
207                                                 Congo
408                             The Lovely Bones: A Novel
541                            The Nanny Diaries: A Novel
1012                                        The Rainmaker
1051                                        SHIPPING NEWS
1195                                        Jurassic Park
1355                                  Message in a Bottle
1387                              A Prayer for Owen Meany
1429                               The Kitchen God's Wife
1777                                         The Alienist
1802                                         The Brethren
1863                                  1st to Die: A Novel
1908    Into Thin Air : A Personal Account of the Mt. ...
1985                                  The Handmaid's Tale
2290    Tuesdays with Morrie: An Old Man, a Young Man,...
2442                                               Sphere
2445          

In [13]:
userID = 53174
topRecommendations = getTopNRecommentationsforUser(userID,N,minRating,ratings_pivot)
print(topRecommendations)
print (ratingsOriginal[(ratingsOriginal['userID'] == userID) & ratingsOriginal['ISBN'].isin(topRecommendations)])

['0345378490', '0312924585', '0440206154', '0553268880', '0345378490', '0971880107', '0446608955', '0446601241', '0375706771', '0345391055', '044651652X', '0440224764', '0345361792', '0385492081', '0452282829', '067976402X', '0312983271', '0553572997', '0440224675', '0440211263', '0385265700', '0553280341', '0671510053', '0345313860', '1400034779', '0345342968', '0786868716', '0446611212', '0060987103', '0156027321']
        userID        ISBN  bookRating
230019   53174  0312924585          10
230049   53174  0345313860           9
230211   53174  0440206154          10
230347   53174  0553268880           9
230353   53174  0553280341           0
230534   53174  0971880107           0


In [14]:
print(books[books['ISBN'].isin(topRecommendations)]["bookTitle"])

26                                            Wild Animus
107                    The Five People You Meet in Heaven
207                                                 Congo
245     Wicked: The Life and Times of the Wicked Witch...
410                                              Hannibal
563                                            Life of Pi
608      The Vampire Lestat (Vampire Chronicles, Book II)
1051                                        SHIPPING NEWS
1129                                       Kiss the Girls
1194                                           Red Dragon
1387                              A Prayer for Owen Meany
1406    The No. 1 Ladies' Detective Agency (Today Show...
1776                                          The Partner
1777                                         The Alienist
1908    Into Thin Air : A Personal Account of the Mt. ...
1922                               Snow Falling on Cedars
2230                                       Fahrenheit 451
2536     The B

In [15]:
userID = 3363
topRecommendations = getTopNRecommentationsforUserAvg(userID,N,minRating,ratings_pivot)
print(topRecommendations)
print (ratingsOriginal[(ratingsOriginal['userID'] == userID) & ratingsOriginal['ISBN'].isin(topRecommendations)])

['0446611212', '0385265700', '080410753X', '0446610038', '0553280341', '0345378490', '0385492081', '0345391055', '0440236673', '0553268880', '0553572997', '0440241073', '014028009X', '0440221471', '0345370775', '0345361792', '044651652X', '0440234743', '0440214041', '0385484518', '080410526X', '0449212602', '0446606812', '0316666343', '0446608955', '0671001795', '0671510053', '067976402X', '0375707972', '0345353145']
       userID        ISBN  bookRating
18025    3363  0316666343           0
18229    3363  0440234743           0
18230    3363  0440236673           0
18254    3363  0446606812           0
18565    3363  080410526X           0


In [16]:
print(books[books['ISBN'].isin(topRecommendations)]["bookTitle"])

18                                          The Testament
207                                                 Congo
408                             The Lovely Bones: A Novel
1051                                        SHIPPING NEWS
1195                                        Jurassic Park
1355                                  Message in a Bottle
1387                              A Prayer for Owen Meany
1429                               The Kitchen God's Wife
1702                                           The Reader
1777                                         The Alienist
1802                                         The Brethren
1863                                  1st to Die: A Novel
1908    Into Thin Air : A Personal Account of the Mt. ...
1922                               Snow Falling on Cedars
1985                                  The Handmaid's Tale
2290    Tuesdays with Morrie: An Old Man, a Young Man,...
2442                                               Sphere
2445          

In [17]:
userID = 53174
topRecommendations = getTopNRecommentationsforUserAvg(userID,N,minRating,ratings_pivot)
print(topRecommendations)
print (ratingsOriginal[(ratingsOriginal['userID'] == userID) & ratingsOriginal['ISBN'].isin(topRecommendations)])

['0553268880', '0345391055', '0446601241', '044651652X', '0345378490', '0345370775', '0440221471', '0446364193', '0553280341', '0345361792', '0345313860', '0671001795', '044021145X', '0446608955', '080410753X', '0440234743', '0446607657', '0312966091', '0971880107', '0440236673', '014028009X', '0449212602', '0804106304', '0446611212', '0440241073', '0440224675', '0671510053', '059035342X', '0345380371', '0452282829']
        userID        ISBN  bookRating
230049   53174  0345313860           9
230252   53174  0446364193           7
230347   53174  0553268880           9
230353   53174  0553280341           0
230534   53174  0971880107           0


In [18]:
print(books[books['ISBN'].isin(topRecommendations)]["bookTitle"])

18                                          The Testament
26                                            Wild Animus
67                                      The Joy Luck Club
207                                                 Congo
410                                              Hannibal
456                                              The Firm
608      The Vampire Lestat (Vampire Chronicles, Book II)
677               Along Came a Spider (Alex Cross Novels)
1051                                        SHIPPING NEWS
1129                                       Kiss the Girls
1195                                        Jurassic Park
1387                              A Prayer for Owen Meany
1429                               The Kitchen God's Wife
1802                                         The Brethren
1860                                           Rising Sun
1985                                  The Handmaid's Tale
2143    Harry Potter and the Sorcerer's Stone (Harry P...
2866          

### As shown above I printed the books rating that was recommended by this system and rated by the user himself, but not considered in the system as they didn't meet the minimum ratings count as discussed the individual rating technique returns better results